### Imports


In [9]:
import requests
import json
from bs4 import BeautifulSoup

2.Send an HTTP Request to the Website

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"

response = requests.request(method="GET", url=url)
response.status_code

3. Parse the HTML Content with opinions

In [ ]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(page_dom.prettify())

4.Extract required data from parsed code

In [ ]:
opinion = page_dom.select_one("div.js_product-review")
print(opinion.prettify())